# Improved Changes

- Boards with a different inital chip count per column is supported
- Model trains againsts multiple different versions of previous other models, improving its accuracy
- Added a goal and began hardcoding more strategies for board to go for goal rather than highest score

In [2]:
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque, defaultdict
from itertools import count, permutations, product, chain
import itertools
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import copy
import pickle
import time
import networkx as nx
from networkx.algorithms import bipartite

# Board Methods

The board will be represented as a 3D array, where each array is a column state, and each token in the column is a 2-item list. The first item is the level the token is on (-1 if the token has been removed) and the second item is whether remover has just selected the token. Also within the list, a token always keeps its position within the column.

Example(board started out with 3 columns and 2 tokens per column): 

<img src="images/ex2.png" width="200">

Representation: [[[2, 0], [1, 1]], [[0, 0], [3, 1]], [[-1, 0], [-1, 0]]]

score_board Method:

This method scores a given board state, crucial for giving feedback to pusher and remover neural network. Right now the scoring method using an exponential model for scoring based on the number of columns times the number of tokens. That way, as an end result, Pusher is incentived to get a token as high up as possible as their main goal as opposed to many tokens at a lower level. However, for now there are drawbacks to this as this could cause pusher to think too much in the short term.

In [3]:
class Board():
    def __init__(self, n, k, goal=10):
        self.board = {}
        self.selected = {}
        self.goal = goal
        for i in range(n):
            self.board[i] = [[0, 0] for j in range(k)] #level, selected:
            #-1 if removed, 0 if not selected, 1 if selected
        self.n = n
        self.k = k
        self.max_score = 0
    
    def game_over(self):
        if self.max_score >= self.goal:
            return True
        for i in range(self.n):
            for level, sel in self.board[i]:
                if level != -1:
                    return False
        return True
    
    def board_sum(self):
        total = 0
        for i in range(self.n):
            total += sum([item[0] for item in self.board[i]])
        return total
    
    def unique(self):
        ans = ""
        for i in range(self.n):
            for level, sel in self.board[i]:
                ans += str(level)
                ans += "s"
                ans += str(sel)
                ans += "s"
        return ans
                          
    def __str__(self): #Prints the board out in a neat fashion
        ans = ""
        for i in range(self.n):
            ans += f"Column {i}: {self.board[i]}\n"
        return ans

In [4]:
#1 if board1 < board2, 0 if board1 = board2, -1 if board1 > board2, 2 if incomparable
def lessThan(board1, board2):
    B = nx.Graph()
    if board1.n != board2.n or board1.k != board2.k:
        return 2
    top_nodes = [i for i in range(board1.n)]
    bottom_nodes = [(board1.n + i) for i in range(board1.n)]
    B.add_nodes_from(top_nodes, bipartite=0)
    B.add_nodes_from(bottom_nodes, bipartite=1)
    connections_less = []
    connections_more = []
    for i in range(board1.n):
        for j in range(board2.n):
            board1_lis = [item[0] for item in board1.board[i]]
            board2_lis = [item[0] for item in board2.board[j]]
            lessThan = True
            greaterThan = True
            for item1, item2 in zip(board1_lis, board2_lis):
                if item1 > item2:
                    lessThan = False
                if item1 < item2:
                    greaterThan = False
            if lessThan:
                connections_less.append((i, board1.n + j))
            if greaterThan:
                connections_more.append((i, board1.n + j))
    B.add_edges_from(connections_less)
    #Obtain the maximum cardinality matching
    my_matching_less = bipartite.matching.hopcroft_karp_matching(B, top_nodes)
    B.remove_edges_from(connections_less)
    B.add_edges_from(connections_more)
    my_matching_more = bipartite.matching.hopcroft_karp_matching(B, top_nodes)
    isLess = (len(my_matching_less) == board1.n * 2)
    isMore = (len(my_matching_more) == board1.n * 2)
    if isMore and isLess:
        return 0
    if isLess:
        return 1
    if isMore:
        return -1
    return 2

# Pusher Methods

In [5]:
N = 6
K = 3
subset_graph = {}
num_graph = {}
index = 0
values = [i for i in range(N * K)]
for i in range(len(values) + 1):
    for subset in itertools.combinations(values, i):
        subset_graph[index] = subset
        num_graph[tuple(subset)] = index
        index += 1
#structure will be subset_graph[num] = subset
#strucutre of num_graph will be num_graph[subset] = num

#Helper method for getting pushers possible moves
def get_subset(state, num):#Given a subset number, gets the subset
    return subset_graph[num]


#ACTUALLY MODIFIES THE BOARD
def make_pusher_board(state, subset):
    index = 0
    for i in range(state.n):
        for j in range(state.k):
            if index in subset and state.board[i][j][0] != -1:
                state.board[i][j][1] = 1
                state.board[i][j][0] += 1
            index += 1
        state.board[i].sort()
    return state



def get_poss(col, offset):
    poss = []
    tokens = {}
    for item in col:
        if item[0] != -1:
            if item[0] not in tokens:
                tokens[item[0]] = [[]]
            lis = copy.deepcopy(tokens[item[0]][-1])
            lis.append(offset)
            tokens[item[0]].append(lis)
        offset += 1
    all_combinations = list(product(*tokens.values()))
    unique_combinations = set()
    for combination in all_combinations:
        flattened = tuple(sorted(chain.from_iterable(combination)))
        unique_combinations.add(flattened)
    final_combinations = [list(comb) for comb in unique_combinations]
    final_combinations.sort(key=lambda x: (len(x), x))
    return final_combinations
    
    
def recur_get_poss(poss, match, prev_cols, depth):
    filled_cols = 0
    for item in prev_cols:
        if item > 0:
            filled_cols += 1 
    maxlen = float('inf')
    if depth in match:
        for item in match[depth]:
            maxlen = min(maxlen, prev_cols[item])
    if depth == len(poss) - 1:
        if filled_cols == 0:
            return [None]
        elif filled_cols == 1:
            if len(poss[depth]) <= 1:
                return [None]
            lis = poss[depth][1:]
            return [item for item in lis if len(item) <= maxlen]
        elif filled_cols >= 2:
            return [item for item in poss[depth] if len(item) <= maxlen]
    elif depth < len(poss) - 1:
        poss_dic = {}
        lengths = set()
        for item in poss[depth]:
            if len(item) <= maxlen:
                lengths.add(len(item))
        for length in lengths:
            poss_dic[length] = recur_get_poss(poss, match, prev_cols + [length], depth + 1)
        ans = []
        for item in poss[depth]:
            if len(item) <= maxlen:
                for item2 in poss_dic[len(item)]:
                    if item2 != None:
                        ans.append(item + item2)
        return ans
        
        

def is_possible_push(state):
    #Get all matching columns
    diff_cols = []
    for i in range(state.n):
        for row, _ in state.board[i]:
            if row != -1:
                diff_cols.append(i)
                break
        if len(diff_cols) == 2:
            break
    if len(diff_cols) == 1:
        offset = state.k * diff_cols[0]
        ans = []
        for row, _ in state.board[diff_cols[0]]:
            if row != -1:
                ans.append(offset)
            offset += 1
        return [num_graph[tuple(ans)]]
    match = {} #dictionary of dependencies (e.g.) can't add something unless something from its dependency is added
    for i in range(state.n):
        for j in range(i + 1, state.n):
            if state.board[i] == state.board[j]:
                if j not in match:
                    match[j] = []
                match[j].append(i)
    poss = {}
    for i in range(state.n):
        poss[i] = get_poss(state.board[i], i * state.k)
    subsets = recur_get_poss(poss, match, [], 0)
    ans = [num_graph[tuple(subset)] for subset in subsets]
    ans.sort()
    return ans
    
def make_move_pusher(state):
    '''
    Makes a move given a pusher neural network. If no network given, then move is random.
    Bound variable is used to add randomness to the process if given a network, if bound is 0 that means no randomness. 
    '''
    poss = is_possible_push(state)
    action = random.choice(poss)
    subset = get_subset(state, action)
    state = make_pusher_board(state, subset)
    return state

# Remover Methods

In [6]:
def is_possible_remove(state): #Given a state, gets all the possible moves for the remover
    poss = []
    visited = set()
    for i in range(state.n):
        for label, sel in state.board[i]:
            if sel == 1:
                val = tuple([item for sublist in state.board[i] for item in sublist])
                if val not in visited:
                    visited.add(val)
                    poss.append(i)
            if sel == 1 and label == state.goal:
                return [i]
    poss = list(dict.fromkeys(poss))
    not_include = set()
    if len(poss) > 1:
        temp_dic = {}
        for item in poss:
            temp_state = copy.deepcopy(state)
            make_remover_board(temp_state, item)
            temp_dic[item] = temp_state
        for i in range(len(poss)):
            for j in range(i + 1, len(poss)):
                score = lessThan(temp_dic[poss[i]], temp_dic[poss[j]])
                if score == 0 or score == -1:
                    not_include.add(poss[i])
                if score == 1:
                    not_include.add(poss[j])
    poss = [item for item in poss if item not in not_include]
    return poss


#METHOD THAT ACTUALLY MODIFIES STATE
def make_remover_board(state, action):
    for j in range(state.k):
        if state.board[action][j][1] == 1:
            state.board[action][j][0] = -1
    for i in range(state.n):
        for j in range(state.k):
            state.board[i][j][1] = 0
            state.max_score = max(state.max_score, state.board[i][j][0])
        state.board[i].sort()
    return state


def make_move_remover(state):#uses the current remover net to make a move for the remover, otherwise it goes random
    poss = is_possible_remove(state)
    action = -1
    if len(poss) == 0:
        action = 0
    else:
        action = random.choice(poss)
    state = make_remover_board(state,action)
    return state
    

# Losing Positions

# Winning Positions

In [35]:
with open('losing_list.pkl', 'rb') as file:
    LOSING = pickle.load(file)

In [32]:
print(len(LOSING))

357


# General Board Methods

In [65]:
def sim_game(n, k, goal):#Simulates a game, printing board state along the way
    state = Board(n, k, goal)
    while state.game_over() == False:
        state = make_move_pusher(state)
        print("Pusher's move: ")
        print(state)
        state = make_move_remover(state)
        print("Remover's move: ")
        print(state)
        print()
    print("GAME OVER")
    print(f"Pusher's Max Score: {state.max_score}")
    print(f"Pusher's Goal: {state.goal}")
    if state.goal <= state.max_score:
        print("Pusher Reached Their Goal")
        return state.max_score, 1 #1 means Pusher wins
    else:
        print("Remover Reached Their Goal")
        return state.max_score, -1 #-1 means Remover wins

def sim_game_no_print(n, k, goal):#Simulates the game without printing the board state
    state = Board(n, k, goal)
    while state.game_over() == False:
        state = make_move_pusher(state)
        state = make_move_remover(state)
    if state.goal <= state.max_score:
        return state.max_score, 1 #1 means Pusher wins
    else:
        return state.max_score, -1 #-1 means Remover wins

    
CURR_BOUND = 200
def minimax(state, isPusher, alpha, beta, og):
    global LOSING
    global CURR_BOUND
#     if isPusher:
#         if state.game_over():
#             return state.max_score
#         for lose_board in LOSING:
#             temp = lessThan(state, lose_board)
#             if temp == 1 or temp == 0:
#                 return -1
    if isPusher:
        bestVal = float('-inf')
        if og:
            print(len(is_possible_push(state)))
        for poss in is_possible_push(state):
            subset = get_subset(state, poss)
            nex = copy.deepcopy(state)
            make_pusher_board(nex, subset)
            value = minimax(nex, False, alpha, beta, False)
            bestVal = max(bestVal, value)
            if og:
                print(bestVal)
            if bestVal >= state.goal:
                return bestVal
            alpha = max(alpha, bestVal)
            if beta <= alpha:
                break
#         if bestVal < state.goal:
#             LOSING.append(state)
#             if len(LOSING) > CURR_BOUND:
#                 print(len(LOSING))
#                 LOSING = prune_losing(LOSING)
#                 print(len(LOSING))
#                 CURR_BOUND = max(CURR_BOUND, len(LOSING) + 40)
        return bestVal
    else:
        bestVal = float('inf')
        for poss in is_possible_remove(state):
            nex = copy.deepcopy(state)
            make_remover_board(nex, poss)
            value = minimax(nex, True, alpha, beta, False)
            bestVal = min(bestVal, value)
            beta = min(beta, bestVal)
            if beta <= alpha:
                break
        return bestVal
#Calling function for first time: minimax(0, 0, true, -inf, +inf)

#1 if board1 < board2, 0 if board1 = board2, -1 if board1 > board2, 2 if incomparable
def prune_losing(losing_boards):
    not_include = set()
    for i in range(len(losing_boards)):
        for j in range(i + 1, len(losing_boards)):
            score = lessThan(losing_boards[i], losing_boards[j])
            if score == 0 or score == 1:
                not_include.add(i)
            if score == -1:
                not_include.add(j)
    final_losing = []
    for i in range(len(losing_boards)):
        if i not in not_include:
            final_losing.append(losing_boards[i])
    return final_losing

# Boards to Win From

In [66]:
lose3 = Board(6, 3, 9)
lose3.board[0] = [[4, 0], [4, 0], [4, 0]]
lose3.board[1] = [[4, 0], [4, 0], [4, 0]]
lose3.board[2] = [[4, 0], [4, 0], [4, 0]]
lose3.board[3] = [[4, 0], [4, 0], [4, 0]]
lose3.board[4] = [[-1, 0], [-1, 0], [-1, 0]]
lose3.board[5] = [[-1, 0], [-1, 0], [-1, 0]]
lose3.max_score = 4
minimax(lose3, True, float('-inf'), float('inf'), True)

31


KeyboardInterrupt: 

In [ ]:
lose4 = Board(6, 3, 9)
lose4.board[0] = [[3, 0], [3, 0], [3, 0]]
lose4.board[1] = [[3, 0], [3, 0], [3, 0]]
lose4.board[2] = [[3, 0], [3, 0], [3, 0]]
lose4.board[3] = [[3, 0], [3, 0], [3, 0]]
lose4.board[4] = [[3, 0], [3, 0], [3, 0]]
lose4.board[5] = [[-1, 0], [-1, 0], [-1, 0]]
lose4.max_score = 3
minimax(lose4, True, float('-inf'), float('inf'), True)

In [62]:
with open('losing_list2.pkl', 'wb') as file:
    pickle.dump(LOSING, file)

In [57]:
print(len(LOSING))

953


In [ ]:
for board in LOSING:
    print(board)

In [ ]:
#Brute forcing all losing states

In [ ]:
#one token higher than the rest, push everything up

In [ ]:
#Work on a better losing database, and time how long each comparison takes

# Generate Losing States

In [ ]:
#Idea: mimic ideal gameplay in minimax, and for each board on this path that is a losing state to add to the graph